In [1]:
from crewai import Agent, Task, Crew
from langgraph.graph import Graph
from langchain.llms import Ollama
from docx import Document
from io import StringIO
import sys

C:\Users\SathyaNarayanan\AppData\Local\Programs\Python\Python312\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(
C:\Users\SathyaNarayanan\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_internal\_generate_schema.py:913: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [2]:
# Initialize LLM
llm = Ollama(base_url='http://localhost:11434', model="llama3.2")

In [3]:
# Load your docx file
doc = Document("Account Payment - WebIPS.docx")

In [4]:
# Extract all paragraphs
text = "\n".join([para.text for para in doc.paragraphs])
# Store in a variable
doc_content = text
# Print to check
print(doc_content)

1. How to Access the Account Payment?
To view the Account Payment screen, you can either:
Search for "Account Payment" using the search bar, or
Navigate through the menu: Menu → Billing → Account Payment.


2. What is the Usage of Account Payment?
From the Account Payment screen, you can:
Reconcile payments for self-pay prescriptions and the Patient Responsible Amount (copay) received from insurance.
Create receipts for credit amounts on the accounts.


3. How to create Account Payment Receipts?
In the Account field, a list of accounts will be displayed along with phone number and address details.
If the account is for a system patient, a "P" badge will appear before the account’s name.
If the account is for a contact person (created from the Contact Master), a "C" badge will appear before the account’s name.

If the account is for a facility, an "F" badge will appear before the account’s name.
Upon selecting an account (responsible for the bill), prescription details such as Bill Serv

In [5]:
# Define Agents
junior_qa = Agent(
    role="Junior Quality Assurance Engineer",
    goal="Write comprehensive test cases covering both happy path and non-happy path scenarios based on user guide content",
    backstory="You are an enthusiastic junior QA engineer who meticulously writes detailed test cases. "
              "You focus on covering all possible scenarios while strictly following the document requirements.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

senior_qa = Agent(
    role="Senior Quality Assurance Lead",
    goal="Validate test cases against the original document content to ensure accuracy, completeness and prevent hallucinations",
    backstory="You are an experienced senior QA lead with years of experience in test case validation. "
              "You have a keen eye for detail and ensure all test cases align perfectly with requirements.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [6]:
# Define Tasks
write_test_cases_task = Task(
    description=f"""
    Based on the following user guide content, write comprehensive test cases:
    
    DOCUMENT CONTENT:
    {doc_content}
    
    REQUIREMENTS:
    1. Write test cases for ALL functionalities mentioned in the document
    2. Each test case must have:
       - Test Case ID
       - Test Case Description
       - Pre-conditions
       - Test Steps (clear, sequential steps)
       - Expected Results
       - Test Type (Happy Path / Non-Happy Path)
    3. Cover both Happy Path (positive scenarios) and Non-Happy Path (negative/error scenarios)
    4. Do NOT add any features not mentioned in the document
    5. Focus on the exact requirements specified
    6. Minimum 3-5 test cases per major functionality
    
    OUTPUT FORMAT:
    For each test case, provide in this exact structure:
    TC_ID: [unique identifier]
    Description: [brief description]
    Pre-conditions: [prerequisites]
    Steps: [numbered steps]
    Expected: [expected results]
    Type: [Happy Path/Non-Happy Path]
    """,
    agent=junior_qa,
    expected_output="Comprehensive test cases covering all document functionalities with both happy and non-happy paths",
    verbose=True
)

validate_test_cases_task = Task(
    description=f"""
    Validate the test cases generated by the Junior QA against the original document content:
    
    ORIGINAL DOCUMENT CONTENT:
    {doc_content}
    
    VALIDATION CRITERIA:
    1. Check if all test cases align with the document requirements
    2. Verify no hallucinations - no features/requirements not in the document
    3. Ensure both happy path and non-happy path scenarios are covered
    4. Validate test steps are logical and achievable
    5. Check expected results match the document specifications
    6. Ensure comprehensive coverage of all mentioned functionalities
    
    For any issues found, provide specific feedback and corrections.
    If test cases are valid, provide confirmation of approval.
    
    Provide your validation report in this format:
    VALIDATION SUMMARY:
    - Total Test Cases Reviewed: [number]
    - Approved: [number]
    - Needs Correction: [number]
    - Key Issues Found: [list]
    - Overall Status: [PASS/FAIL WITH COMMENTS]
    """,
    agent=senior_qa,
    expected_output="Validation report with approved test cases or specific feedback for corrections",
    verbose=True
)

In [7]:
# Create Crew for sequential execution
qa_crew = Crew(
    agents=[junior_qa, senior_qa],
    tasks=[write_test_cases_task, validate_test_cases_task],
    verbose=True
)

In [8]:
# Custom output capture that writes to both console and file in real-time
class RealTimeCapture:
    def __init__(self, filename="agent_cot_output.txt"):
        self.original_stdout = sys.stdout
        self.capture_file = open(filename, 'w', encoding='utf-8')
        self.buffer = StringIO()
        
    def write(self, text):
        # Write to original stdout (console)
        self.original_stdout.write(text)
        # Write to file immediately
        self.capture_file.write(text)
        self.capture_file.flush()  # Force write to disk
        # Also buffer for later use
        self.buffer.write(text)
        
    def flush(self):
        self.original_stdout.flush()
        self.capture_file.flush()
        
    def close(self):
        self.capture_file.close()
        
    def get_captured_text(self):
        return self.buffer.getvalue()

In [9]:
# LangGraph implementation for workflow control
def create_qa_workflow():
    workflow = Graph()
    
    def junior_qa_node(state):
        print("🧪 Junior QA: Writing test cases...")
        test_cases = f"""
        Test Cases generated for document of {len(doc_content)} characters.
        This would contain actual test cases in production.
        """
        return {"test_cases": test_cases, "document": doc_content}
    
    def senior_qa_node(state):
        print("🔍 Senior QA: Validating test cases...")
        test_cases = state.get("test_cases", "")
        validation_result = f"""
        Validation completed for test cases.
        Original document: {len(state.get('document', ''))} characters
        Test cases generated: {len(test_cases)} characters
        """
        return {"validation_result": validation_result, "test_cases": test_cases}
    
    def final_node(state):
        print("📋 Final: Compiling results...")
        final_output = f"""
        FINAL TEST CASE REPORT
        =====================
        
        Document Analysis: {len(state.get('document', ''))} characters
        Test Cases Generated: {state.get('test_cases', '')}
        Validation Results: {state.get('validation_result', '')}
        
        Status: COMPLETED
        """
        return {"final_output": final_output}
    
    # Add nodes to workflow
    workflow.add_node("junior_qa", junior_qa_node)
    workflow.add_node("senior_qa", senior_qa_node)
    workflow.add_node("final", final_node)
    
    # Define workflow sequence
    workflow.set_entry_point("junior_qa")
    workflow.add_edge("junior_qa", "senior_qa")
    workflow.add_edge("senior_qa", "final")
    workflow.set_finish_point("final")
    return workflow

In [10]:
# Main execution function
def generate_test_cases():
    print("🚀 Starting Test Case Generation System...")
    print(f"Document Content Length: {len(doc_content)} characters")
    
    # Create real-time capture
    capture = RealTimeCapture("agent_cot_output.txt")
    
    # Redirect stdout to our capture class
    old_stdout = sys.stdout
    sys.stdout = capture
    
    try:
        # Method 1: Using CrewAI (Primary method)
        print("\n" + "="*60)
        print("METHOD 1: USING CREWAI AGENTS")
        print("="*60)
        
        crew_result = qa_crew.kickoff()
        print("\n🎯 CREWAI RESULT:")
        print(crew_result)
        
        # Method 2: Using LangGraph workflow
        print("\n" + "="*60)
        print("METHOD 2: USING LANGGRAPH WORKFLOW")
        print("="*60)
        
        workflow = create_qa_workflow()
        app = workflow.compile()
        
        # Execute the workflow
        initial_state = {"document": doc_content}
        graph_result = app.invoke(initial_state)
        
        print("\n🎯 LANGGRAPH RESULT:")
        print(graph_result.get("final_output", "No output generated"))
        
        return crew_result, graph_result
        
    finally:
        # Restore stdout and close capture
        sys.stdout = old_stdout
        capture.close()

In [11]:
# Run the system
if __name__ == "__main__":
    try:
        crew_result, graph_result = generate_test_cases()
        
        # Save final results to separate file
        with open("test_cases_final_results.txt", "w") as f:
            f.write("CREWAI RESULT:\n")
            f.write(str(crew_result))
            f.write("\n\n" + "="*50 + "\n\n")
            f.write("LANGGRAPH RESULT:\n")
            f.write(str(graph_result))
        
        print("\n✅ Test case generation completed!")
        print("📁 Complete Chain of Thought saved to: agent_cot_output.txt")
        print("📁 Final results saved to: test_cases_final_results.txt")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Please check your Ollama installation and model availability.")

🚀 Starting Test Case Generation System...
Document Content Length: 4393 characters

METHOD 1: USING CREWAI AGENTS
 [DEBUG]: == Working Agent: Junior Quality Assurance Engineer
 [INFO]: == Starting Task: 
    Based on the following user guide content, write comprehensive test cases:
    
    DOCUMENT CONTENT:
    1. How to Access the Account Payment?
To view the Account Payment screen, you can either:
Search for "Account Payment" using the search bar, or
Navigate through the menu: Menu → Billing → Account Payment.


2. What is the Usage of Account Payment?
From the Account Payment screen, you can:
Reconcile payments for self-pay prescriptions and the Patient Responsible Amount (copay) received from insurance.
Create receipts for credit amounts on the accounts.


3. How to create Account Payment Receipts?
In the Account field, a list of accounts will be displayed along with phone number and address details.
If the account is for a system patient, a "P" badge will appear before the accoun